In [3]:
import pandas as pd
import ccxt
from tqdm import tqdm
import time
import random


In [29]:
binance = ccxt.binance()


pairs = [pair for pair in binance.load_markets() if pair.endswith("USDT")]

print("number of pairs :", len(pairs))

#percentage = float(input("Enter the percentage of data you want to fetch: "))
percentage =100
print(f"Fetching {percentage}% of data...")
if percentage < 1 or percentage > 100:

    raise ValueError("Percentage must be between 1 and 100")

select_pairs = random.sample(pairs, int(len(pairs) * percentage / 100))

timeframes = ["1d"]
all_data = []

# Calculate the start date based on the number of candles we want
for timeframe in timeframes:
    print(f"Fetching data for {timeframe} timeframe...")
    for pair in tqdm(select_pairs):
        try:
            # Fetch last 10 candles directly
            candles = binance.fetch_ohlcv(pair, timeframe, limit=15)

            for candle in candles:
                timestamp, open_, high, low, close, volume = candle
                all_data.append(
                    {
                        "pair": pair,
                        "timeframe": timeframe,
                        "timestamp": pd.to_datetime(timestamp, unit="ms"),
                        "open": open_,
                        "high": high,
                        "low": low,
                        "close": close,
                        "volume": volume,
                    }
                )

            time.sleep(1)  # Rate limiting

        except Exception as e:
            print(f"Error fetching data for {pair} on {timeframe}: {e}")

# Convert to DataFrame with timeframe information
df = pd.DataFrame(all_data)

print(df)


number of pairs : 888
Fetching 100% of data...
Fetching data for 1d timeframe...


  9%|▉         | 78/888 [01:39<13:07,  1.03it/s]

Error fetching data for BTCST/USDT:USDT on 1d: binance {"code":-1122,"msg":"Invalid symbol status."}


100%|██████████| 888/888 [19:03<00:00,  1.29s/it]

            pair timeframe  timestamp     open     high      low    close  \
0      PROS/USDT        1d 2024-11-27  0.68010  0.75400  0.67060  0.70900   
1      PROS/USDT        1d 2024-11-28  0.70900  0.76000  0.68030  0.73130   
2      PROS/USDT        1d 2024-11-29  0.73120  0.78000  0.71000  0.72170   
3      PROS/USDT        1d 2024-11-30  0.72200  0.74300  0.71340  0.72120   
4      PROS/USDT        1d 2024-12-01  0.72120  0.72940  0.69990  0.70530   
...          ...       ...        ...      ...      ...      ...      ...   
13148  ARDR/USDT        1d 2024-12-07  0.12812  0.12812  0.12272  0.12637   
13149  ARDR/USDT        1d 2024-12-08  0.12631  0.12640  0.12119  0.12612   
13150  ARDR/USDT        1d 2024-12-09  0.12623  0.12636  0.10271  0.10856   
13151  ARDR/USDT        1d 2024-12-10  0.10871  0.10967  0.09374  0.10202   
13152  ARDR/USDT        1d 2024-12-11  0.10202  0.10972  0.09590  0.10770   

           volume  
0       9165426.0  
1       8045553.0  
2       8388380

In [30]:

pairs = df["pair"].unique()


def buy_or_sell_candell(id, df) -> bool:
    return df.loc[id]["close"] > df.loc[id]["open"]

print(pairs)

['PROS/USDT' 'ADX/USDT' 'AVAX/USDT:USDT' 'UTK/USDT' 'SPX/USDT:USDT'
 'ACE/USDT:USDT' 'ACH/USDT' 'TIA/USDT' 'CYBER/USDT:USDT' 'REP/USDT'
 'PHB/USDT' 'KNC/USDT:USDT' 'GHST/USDT' 'LOKA/USDT:USDT' 'RPL/USDT'
 'VIB/USDT' 'SYN/USDT:USDT' 'MINA/USDT' 'BNT/USDT:USDT' 'EDU/USDT:USDT'
 'SAFE/USDT:USDT' 'TIA/USDT:USDT' 'KP3R/USDT' 'BONK/USDT'
 'THETA/USDT:USDT' 'BAND/USDT' 'STORM/USDT' 'KOMA/USDT:USDT' 'PUNDIX/USDT'
 'SXP/USDT:USDT' 'DNT/USDT' 'VANRY/USDT:USDT' 'BLUR/USDT' 'SCR/USDT:USDT'
 'FIDA/USDT' 'BNB/USDT' 'MDX/USDT' 'TROY/USDT:USDT' 'TLM/USDT' 'MC/USDT'
 '1000PEPE/USDT:USDT' 'CAKE/USDT:USDT' 'UST/USDT' 'XNO/USDT'
 '1000BONK/USDT:USDT' 'ATM/USDT' 'JUP/USDT' 'STG/USDT:USDT'
 'HIFI/USDT:USDT' 'NOT/USDT:USDT' 'XLMDOWN/USDT' 'BTS/USDT'
 'JASMY/USDT:USDT' 'USDS/USDT' 'BCHDOWN/USDT' 'FDUSD/USDT' 'CHZ/USDT'
 'BETA/USDT' 'RARE/USDT:USDT' 'AVAX/USDT' 'AR/USDT:USDT' 'ONG/USDT:USDT'
 'ALICE/USDT:USDT' 'OGN/USDT' 'BNBBEAR/USDT' 'RONIN/USDT:USDT'
 'BNB/USDT:USDT' 'AKRO/USDT' 'NEIRO/USDT:USDT' 'LEVER/USD

In [ ]:
for pair in pairs:
    for timeframe in timeframes:
        df_pair_timeframe = df[
            (df["pair"] == pair) & (df["timeframe"] == timeframe)
        ].copy()
        
        if df_pair_timeframe.empty:
            print(f"No data for pair {pair} and timeframe {timeframe}")
            continue
        
        df_pair_timeframe = df_pair_timeframe.reset_index(drop=True)
        
        # Avoid division by zero
        if (df_pair_timeframe["low"] == 0).any():
            print(f"Zero low value found for pair {pair} and timeframe {timeframe}")
            continue
        
        df_pair_timeframe["high_low_range"] = (
            (df_pair_timeframe["high"] - df_pair_timeframe["low"])
            * 100
            / df_pair_timeframe["low"]
        )
        
        # Check if high_low_range has valid data
        if df_pair_timeframe["high_low_range"].empty:
            print(f"No high_low_range data for pair {pair} and timeframe {timeframe}")
            continue
        
        bigger_candel = df_pair_timeframe["high_low_range"].idxmax()
        
        # Determines whether the "bigger candle" is a green (buy) or red (sell) candle.
        
        # Args:
        #     bigger_candel (int): The index of the "bigger candle" in the DataFrame.
        #     df_pair_timeframe (pandas.DataFrame): The DataFrame containing the price data for the current pair and timeframe.
        
        # Returns:
        #     bool: True if the "bigger candle" is a green (buy) candle, False otherwise.
        green_big_candel = buy_or_sell_candell(bigger_candel, df_pair_timeframe)
        
        if not green_big_candel:
            continue
        

        next_candels = df_pair_timeframe.loc[bigger_candel + 1 :]
        
        if next_candels.empty:
            #print(f"No candles after index {bigger_candel} for pair {pair}")
            continue
        
        price_high = df_pair_timeframe.loc[bigger_candel, "high"]
        price_low = df_pair_timeframe.loc[bigger_candel, "low"]
        price_50 = (price_high + price_low) / 2
        price_3 = (price_high + price_low) * 0.97
        price_5 = (price_high + price_low) * 0.95

        exceeds_high = any(
            candle_high > price_high for candle_high in next_candels["high"]
        )
        below_50 = any(candle_low < price_50 for candle_low in next_candels["low"])

        last_candle_high = next_candels.tail(1)["high"].values[0]

        between_5_0 = price_5 <= last_candle_high <= price_high
        
        price_30 = (price_high + price_low) * 0.7
        
        between_30_0 = price_30 <= last_candle_high <= price_high

        if (
            not exceeds_high
            and not below_50
            #and between_5_0
            and between_30_0
        ):
            print(
                f"Pair: {pair} | Timeframe: {timeframe} | Price high: {price_high} | Number of candles: {len(next_candels)}"
            )


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2243067940.py, line 71)